In [1]:
#Import libraries
#Set seed for random number generator to create reproducible results
library(ISLR)
set.seed(1)
train=sample(392,196)

Warning message:
"package 'ISLR' was built under R version 3.6.1"

In [4]:
#?sample

In [5]:
#Use subset option in LM to fit regression using only training set
lm.fit=lm(mpg~horsepower,data=Auto,subset=train)

In [7]:
#Use predict function to estimate response for 392 observ
#Mean function to calculate MSE of 196 observations
# "negative train" selects observations not in the training set
attach(Auto)
mean((mpg-predict(lm.fit,Auto))[-train]^2)

The following objects are masked from Auto (pos = 3):

    acceleration, cylinders, displacement, horsepower, mpg, name,
    origin, weight, year



[1] 23.26601

<b>Mean Squared Error = 23.26<b>

In [9]:
# poly() function to estimate  test error for quadratic and cubic regression
lm.fit2=lm(mpg~poly(horsepower,2), data=Auto,subset=train)
mean((mpg-predict(lm.fit2,Auto))[-train]^2)

[1] 18.71646

In [11]:
lm.fit3=lm(mpg~poly(horsepower,3),data=Auto,subset=train)
mean((mpg-predict(lm.fit3,Auto))[-train]^2)

[1] 18.79401

Error rates are 19.82 and 19.78.

Choosing a different training set will result in different errors on thevalidation set.

In [14]:
#Linear Model
set.seed(2)
train=sample(392,196)
lm.fit=lm(mpg~horsepower,subset=train)
mean((mpg-predict(lm.fit,Auto))[-train]^2)

[1] 25.72651

In [15]:
#Cubic Model
lm.fit2=lm(mpg~poly(horsepower,2),data=Auto,subset=train)
mean((mpg-predict(lm.fit2,Auto))[-train]^2)

[1] 20.43036

In [16]:
#Quadratic Model
lm.fit3=lm(mpg~poly(horsepower,3),data=Auto,subset=train)
mean((mpg-predict(lm.fit3,Auto))[-train]^2)

[1] 20.38533

Quadratic function performs better than linear regression in predicting Horsepower by MPG in the Auto dataset.

## Leave-One-Out Cross-Validation

In [17]:
glm.fit=glm(mpg~horsepower,data=Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [18]:
lm.fit=lm(mpg~horsepower,data=Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [19]:
library(boot)
glm.fit=glm(mpg~horsepower,data=Auto)
cv.err=cv.glm(Auto,glm.fit)
cv.err$delta

[1] 24.23151 24.23114

In [21]:
cv.error=rep(0,5)
for (i in 1:5){
    glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
    cv.error[i]=cv.glm(Auto,glm.fit)$delta[1]
    }
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

## k-Fold Cross-Validation

In [22]:
set.seed(17)
cv.error.10=rep(0,10)
for (i in 1:10){
    glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
    cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1]
    }
cv.error.10

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

## Bootstrap

In [24]:
alpha.fn=function(data,index){
    X=data$X[index]
    Y=data$Y[index]
    return((var(Y)-cov(X,Y))/var(X)+var(Y)-2*cov(X,Y))
    }

In [25]:
alpha.fn(Portfolio,1:100)

[1] 0.6596797

In [26]:
set.seed(1)
alpha.fn(Portfolio,sample(100,100,replace=T))

[1] 1.077955

In [27]:
boot(Portfolio,alpha.fn,R=1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.6596797 0.0009365922   0.3082678

In [29]:
boot.fn=function(data,index)
    return(coef(lm(mpg~horsepower,data=data,subset=index)))
boot.fn(Auto,i:392)

(Intercept)  horsepower 
 40.5952783  -0.1651005